### Here is the code of last submission,related explanations refer to the document.

In [38]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
import xgboost as xgb
from sklearn import metrics

In [39]:
#read data
data_train=pd.read_csv('data_train\data_train.csv')
data_test=pd.read_csv('data_test\data_test.csv')
data_train.drop(['Unnamed: 0'],axis=1,inplace=True)
data_test.drop(['Unnamed: 0'],axis=1,inplace=True)
data_train.shape,data_test.shape

((814262, 11), (202937, 11))

In [40]:
#regard (x_lower,y_lower) as coordinate origin(0,0) pan all points
x_upper = 3770901.5068
x_lower = 3750901.5068
y_upper = -19208905.6133 
y_lower = -19268905.6133     #four boundary points
d_x=x_upper-x_lower
d_y=y_upper-y_lower          #boundary distance
def transform_data(data):
    df=data.copy()
    df['x_entry']=x_lower-df['x_entry']
    df['y_entry']=y_lower-df['y_entry']
    df['x_exit']=x_lower-df['x_exit']
    df['y_exit']=y_lower-df['y_exit']
    df['time_entry']=pd.to_datetime(df['time_entry'])
    df['time_exit']=pd.to_datetime(df['time_exit'])
    return df
data_train=transform_data(data_train)
data_test =transform_data(data_test)

In [41]:
#define some helper functions
def is_center(x,d):
    if x<=0 and d+x>=0:
        return 1
    else:
        return 0
def time_diff(x):
    t_x=np.array(x)
    return (t_x[-1]-t_x[0])/np.timedelta64(1, 'm')  
def dist_diff(x):
    d_x=np.array(x)
    return (d_x[-1]-d_x[0])
def bigger_zero(x):
    if x>0:
        return 1
    else:
        return 0

In [42]:
#Separate the dataset according to whether the time interval of the last record is 0(stationary)
def is_zero_dur(data):
    hash_group=data.groupby(['hash'])
    endtimedelta=(hash_group['time_exit'].apply(lambda x:np.array(x)[-1])-
                  hash_group['time_entry'].apply(lambda x:np.array(x)[-1]))/np.timedelta64(1, 'm') #last time duration
    p_time_duration=endtimedelta.to_frame(name='p_time_dur')
    data=pd.merge(data,p_time_duration,left_on='hash',right_index=True,how='inner')
    data_z=data[data.p_time_dur==0]
    data_nz=data[data.p_time_dur!=0]
    return data_z,data_nz
data_train_z,data_train_nz=is_zero_dur(data_train)
data_test_z,data_test_nz=is_zero_dur(data_test)
data_train_z.shape,data_train_nz.shape,data_test_z.shape,data_test_nz.shape

((391381, 12), (422881, 12), (98349, 12), (104588, 12))

In [43]:
def zero_dur_prediction(data):
    df=data.copy()
    hash_group=df.groupby(['hash'])
    endstart_x=hash_group['x_entry'].apply(lambda x:np.array(x)[-1]) #start location x
    endstart_y=hash_group['y_entry'].apply(lambda x:np.array(x)[-1]) #start location y
    end_x=endstart_x.to_frame(name='end_x')
    end_y=endstart_y.to_frame(name='end_y')
    df=pd.merge(df,end_x,left_on='hash',right_index=True,how='inner')
    df=pd.merge(df,end_y,left_on='hash',right_index=True,how='inner')
    df.loc[:,'is_in_center_x']=df.loc[:,'end_x'].apply(lambda x:is_center(x,d_x))
    df.loc[:,'is_in_center_y']=df.loc[:,'end_y'].apply(lambda y:is_center(y,d_y))
    df['label']= np.array(df['is_in_center_x']) & np.array(df['is_in_center_y'])
    return df

In [44]:
dftrain_z=zero_dur_prediction(data_train_z)
dftest_z=zero_dur_prediction(data_test_z)

In [45]:
dframe_test_z=dftest_z.drop(['trajectory_id','time_entry',
                  'time_exit','vmax','vmin','vmean','x_entry','y_entry',
                  'x_exit','y_exit'],axis=1)
dframe_test_z=dframe_test_z.drop_duplicates()
y_pred_z=dframe_test_z['label']
gb_z=dftest_z.groupby(['hash'])
last_id_z=gb_z['trajectory_id'].apply(lambda x:np.array(x)[-1])
result_z=pd.Series(y_pred_z.values,index=last_id_z.values)
result_z.to_frame(name='center_city')
result_z

traj_000506a39775e5bca661ac80e3f466eb_29_5     1
traj_0006535be25bb52dd06983447880c964_5_12     0
traj_0006f84bb33ec929d1cda7686f861d0a_31_3     1
traj_00093ae562586aed0e053b8431e8ace4_23_10    0
traj_000c739e444a70e1804d757a0580caaa_31_3     1
traj_000d6fba12a7d06defe90e3c9162c11e_29_11    1
traj_002447d99e8f6c3bc6c1329140979a91_25_7     0
traj_002531dea45cd4059e92099630ebf775_3_10     0
traj_0026d48a63f219dc359f1131805909bf_31_5     0
traj_002f7d1d09d2af27ee5a6d0fa4e91c96_29_8     1
traj_0032262ce8e9e5e3750f306a2ace8d4a_25_4     1
traj_0033b16d18ce91c89eafacb1cb3a5a44_11_4     0
traj_00390d9128b5d9c7ba1b32c8babb6c24_11_2     0
traj_003d9b2742ddcd1a37717c524135b3d4_3_2      0
traj_003ee2f1ea8002e540b9d4e089fce3f4_9_12     1
traj_0041098d83629d777e85d77974549fe0_11_11    0
traj_004227838e54f53e05b759abd5e44315_15_20    0
traj_004819c7befda3482cff11aba12dcee5_29_3     0
traj_004dff8e9bfa70ac0b87fb7350616ced_15_3     0
traj_0054f50e5b65759e196eeff941043cdf_11_6     1
traj_0058921b1ac097d

In [46]:
def id_count(data):
    df=data.copy()
    hgroup=data.groupby(['hash'])
    id_count=hgroup['trajectory_id'].count()
    id_count=id_count.to_frame(name='id_count')
    data=pd.merge(data,id_count,left_on='hash',right_index=True,how='inner')
    data_id1=data[data['id_count']==1]
    data_id=data[data['id_count']>1]
    return data_id1,data_id

In [47]:
data_train_nz_id1,data_train_nz_id=id_count(data_train_nz)
data_test_nz_id1,data_test_nz_id=id_count(data_test_nz)
data_train_nz_id1.shape,data_train_nz_id.shape,data_test_nz_id1.shape,data_test_nz_id.shape

((813, 13), (422068, 13), (184, 13), (104404, 13))

In [48]:
def featExtraction(data):
    df=data.copy()
    df.loc[:,'is_in_center_x']=df.loc[:,'x_entry'].apply(lambda x:is_center(x,d_x))
    df.loc[:,'is_in_center_y']=df.loc[:,'y_entry'].apply(lambda y:is_center(y,d_y))
    df.loc[:,'start_hour']=df.loc[:,'time_entry'].dt.hour
    df.loc[:,'end_dx_upper']=df.loc[:,'x_entry']-d_x
    df.loc[:,'end_dy_upper']=df.loc[:,'y_entry']-d_y
    df=df[['p_time_dur','x_entry','y_entry','vmin','vmean','vmax',
           'is_in_center_x','is_in_center_y','start_hour','end_dx_upper','end_dy_upper','x_exit','y_exit']]
    return df
df_train_id1=featExtraction(data_train_nz_id1)
df_test_id1=featExtraction(data_test_nz_id1)
df_train_id1.shape,df_test_id1.shape

((813, 13), (184, 13))

In [51]:
df_train_id1['label_x']=df_train_id1.loc[:,'x_exit'].apply(lambda x:is_center(x,d_x))
df_train_id1['label_y']=df_train_id1.loc[:,'y_exit'].apply(lambda y:is_center(y,d_y))
df_train_id1['label']= np.array(df_train_id1['label_x']) & np.array(df_train_id1['label_y'])
X_id=df_train_id1[['p_time_dur','x_entry','y_entry','vmean','is_in_center_x','is_in_center_y','start_hour','end_dx_upper','end_dy_upper']]
y_id=df_train_id1['label']
X_id_t=df_test_id1[['p_time_dur','x_entry','y_entry','vmean','is_in_center_x','is_in_center_y','start_hour','end_dx_upper','end_dy_upper']]

In [52]:
model_id=xgb.XGBClassifier(n_estimators=20,
                         max_depth=5,
                         learning_rate=0.05,
                         gamma=0.1,
                         early_stopping_rounds=5)
model_id.fit(X_id_train,y_id_train,eval_metric='error',eval_set=[(X_id_test,y_id_test)],verbose=True)

[0]	validation_0-error:0.098039
[1]	validation_0-error:0.098039
[2]	validation_0-error:0.098039
[3]	validation_0-error:0.098039
[4]	validation_0-error:0.098039
[5]	validation_0-error:0.098039
[6]	validation_0-error:0.098039
[7]	validation_0-error:0.112745
[8]	validation_0-error:0.102941
[9]	validation_0-error:0.102941
[10]	validation_0-error:0.102941
[11]	validation_0-error:0.098039
[12]	validation_0-error:0.098039
[13]	validation_0-error:0.098039
[14]	validation_0-error:0.098039
[15]	validation_0-error:0.098039
[16]	validation_0-error:0.098039
[17]	validation_0-error:0.093137
[18]	validation_0-error:0.098039
[19]	validation_0-error:0.098039


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=5, gamma=0.1,
       learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=20, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
y_id_pred_te=model_id.predict(X_id_test)
y_id_pred_tr=model_id.predict(X_id_train)
accuracy_rate_test=metrics.f1_score(y_id_test,y_id_pred_te)
accuracy_rate_train=metrics.f1_score(y_id_train,y_id_pred_tr)
accuracy_rate_test,accuracy_rate_train

(0.59999999999999998, 0.67039106145251393)

In [54]:
tar_id=data_test_nz_id1['trajectory_id']
y_id_t=model_id.predict(X_id_t)
result_1=pd.Series(y_id_t,index=tar_id.values)
result_1.to_frame(name='city_center')
result_1

traj_02968210bf5ec87bf8c9da2cb2058c75_29_0    0
traj_03664dd70a14108154d13ed8d2218cd0_25_0    0
traj_03e0f7b02955923917997a6e24b833f0_11_0    0
traj_0444ef9633c78877d07d82effb424248_15_0    0
traj_04fd1bb8290c9a33442edcb6efc18b79_3_0     0
traj_054fc59c279e027f9d53f5ed9bbd125b_25_0    0
traj_0780fbd2ed845ab556a63551f28b30c4_5_1     0
traj_07f983016a4726dc3d31524016953504_29_0    0
traj_0867ee19f1fb8ffc85bd10cda6cbe331_19_1    0
traj_09c5210171d858eee8f1e23a906fdf7e_3_0     0
traj_0c0e2832b69ba001ea1feabd51cb361f_1_0     0
traj_0cc501e69d381687cba1ff4c57a92109_11_0    0
traj_131fe492ff3c89a7294567de9810d745_15_0    0
traj_13d5819df8c1024caffc5224c8464589_25_0    0
traj_14ba47c2fd02768b55b95c8d115f4ba4_5_1     0
traj_1621733c5c0d284d3562bf4d9d99223e_19_0    0
traj_192aa916c69563bb9b07fae2271a2ba9_3_1     0
traj_1932382e35d452d07552c2c041e86f0f_25_4    1
traj_1964480b6c4e746589e885e7637735df_5_0     0
traj_1be351fe4db5b48458120470ec30032c_25_0    0
traj_243b89183dcc636c2dfbe9bfcaa5dea2_19

In [55]:
def feature_engineering(data):
    df=data.copy()
    hash_group=df.groupby(['hash'])
    difftime=hash_group['time_entry'].apply(lambda x:time_diff(x)) #total time interval
    dist_x=hash_group['x_entry'].apply(lambda x:dist_diff(x)) #distance of direction x
    dist_y=hash_group['y_entry'].apply(lambda y:dist_diff(y)) #distance of direction y
    
    endstart_x=hash_group['x_entry'].apply(lambda x:np.array(x)[-1]) #last start location x
    endstart_y=hash_group['y_entry'].apply(lambda x:np.array(x)[-1]) #last start location y
    endtimedelta=(hash_group['time_exit'].apply(lambda x:np.array(x)[-1])-
               hash_group['time_entry'].apply(lambda x:np.array(x)[-1]))/np.timedelta64(1, 'm') #last time interval
    
    start_time_hour=hash_group['time_entry'].apply(lambda x:np.array(x.dt.hour)[0])
    
    vmax=hash_group['vmax'].apply(lambda x:x.max())
    vmin=hash_group['vmin'].apply(lambda x:x.min())
    vmean=hash_group['vmean'].apply(lambda x:x.mean())
    
    
    #first start location
    first_start_x=hash_group['x_entry'].apply(lambda x:np.array(x)[0])
    first_start_y=hash_group['y_entry'].apply(lambda x:np.array(x)[0])
    first_exit_x=hash_group['x_exit'].apply(lambda x:np.array(x)[0])
    first_exit_y=hash_group['y_exit'].apply(lambda x:np.array(x)[0])
    first_time_diff= (hash_group['time_entry'].apply(lambda x:np.array(x)[0])-
                     hash_group['time_exit'].apply(lambda x:np.array(x)[0]))/np.timedelta64(1, 'm')
    
    # distance between first start location and boundary
    first_dist_upperx=d_x - first_start_x
    first_dist_uppery=d_y - first_start_y
    
    # distance between last start location and boundary
    endstart_dist_upperx=d_x - endstart_x
    endstart_dist_uppery=d_y - endstart_y
    
    #Second last record
    pre2_x=hash_group['x_exit'].apply(lambda x:np.array(x)[-2])
    pre2_y=hash_group['y_exit'].apply(lambda x:np.array(x)[-2])
    pre2_x_dist=hash_group['x_exit'].apply(lambda x:np.array(x)[-2])-hash_group['x_entry'].apply(lambda x:np.array(x)[-2])
    pre2_y_dist=hash_group['y_exit'].apply(lambda x:np.array(x)[-2])-hash_group['y_entry'].apply(lambda x:np.array(x)[-2])
    pre2_diff=(hash_group['time_exit'].apply(lambda x:np.array(x)[-2])-hash_group['time_entry'].apply(lambda x:np.array(x)[-2]))/np.timedelta64(1, 'm')
    v2_x=pre2_x_dist/pre2_diff
    v2_y=pre2_y_dist/pre2_diff
        
    feat=[difftime,dist_x,dist_y,endstart_x,
          endstart_y,endtimedelta,start_time_hour,vmax,vmin,vmean,v2_x,v2_y,
          first_start_x,first_start_y,first_exit_x,first_exit_y,first_time_diff,pre2_x,pre2_y,pre2_x_dist,pre2_y_dist,
         first_dist_upperx,first_dist_uppery,endstart_dist_upperx,endstart_dist_uppery]
    for i,x in enumerate(feat):
        feats=x.to_frame(name='feature_'+str(i+1))
        df=pd.merge(df,feats,left_on='hash',right_index=True,how='inner')
    
    df['vel_x']=df['feature_2']/df['feature_1']
    df['vel_y']=df['feature_3']/df['feature_1']
    
    #  distance and speed of first interval
    df.loc[:,'first_dist_x']=df.loc[:,'feature_15'] - df.loc[:,'feature_13']
    df.loc[:,'first_dist_y']=df.loc[:,'feature_16'] - df.loc[:,'feature_14']
    df.loc[:,'first_vx']=df.loc[:,'first_dist_x']/df.loc[:,'feature_17']
    df.loc[:,'first_vy']=df.loc[:,'first_dist_y']/df.loc[:,'feature_17']
    
    #last interval speed
    delta1t= (hash_group['time_entry'].apply(lambda x:np.array(x)[-1])-hash_group['time_exit'].apply(lambda x:np.array(x)[-2]))/np.timedelta64(1, 'm')
    pre_distx=hash_group['x_entry'].apply(lambda x:np.array(x)[-1])-hash_group['x_exit'].apply(lambda x:np.array(x)[-2])
    pre_disty=hash_group['y_entry'].apply(lambda x:np.array(x)[-1])-hash_group['y_exit'].apply(lambda x:np.array(x)[-2])
    pre_vel_x=pre_distx/delta1t
    pre_vel_y=pre_disty/delta1t
    df2_feat1=pre_vel_x.to_frame(name='pre_v1_x')
    df2_feat2=pre_vel_y.to_frame(name='pre_v1_y')
    df=pd.merge(df,df2_feat1,left_on='hash',right_index=True,how='inner')
    df=pd.merge(df,df2_feat2,left_on='hash',right_index=True,how='inner')
    
    #angle
    tanh=pre_vel_y/pre_vel_x
    theta=np.arctan(tanh)
    df_feat3=theta.to_frame(name='theta')
    df=pd.merge(df,df_feat3,left_on='hash',right_index=True,how='inner')
    
    #pre-interval is stationary or not
    df.loc[:,'x_zero_prevel']=df.loc[:,'pre_v1_x'].apply(lambda x:bigger_zero(x))
    df.loc[:,'y_zero_prevel']=df.loc[:,'pre_v1_y'].apply(lambda x:bigger_zero(x))
    
    #start position is in the center or not
    df.loc[:,'start_in_center_x']=df.loc[:,'feature_13'].apply(lambda x:is_center(x,d_x))
    df.loc[:,'start_in_center_y']=df.loc[:,'feature_14'].apply(lambda y:is_center(y,d_y))
    
    #exit position is in the center or not
    df.loc[:,'is_in_center_x']=df.loc[:,'feature_4'].apply(lambda x:is_center(x,d_x))
    df.loc[:,'is_in_center_y']=df.loc[:,'feature_5'].apply(lambda y:is_center(y,d_y))
    return df

In [56]:
df_train_id=feature_engineering(data_train_nz_id)
df_test_id =feature_engineering(data_test_nz_id)
df_train_id.shape,df_test_id.shape

((422068, 53), (104404, 53))

In [57]:
#train_dataset label
hash_group=df_train_id.groupby(['hash'])
endexit_x=hash_group['x_exit'].apply(lambda x:np.array(x)[-1])
endexit_y=hash_group['y_exit'].apply(lambda x:np.array(x)[-1])
exit_x=endexit_x.to_frame(name='label_x_d')
exit_y=endexit_y.to_frame(name='label_y_d')
df_train_id=pd.merge(df_train_id,exit_x,left_on='hash',right_index=True,how='inner')
df_train_id=pd.merge(df_train_id,exit_y,left_on='hash',right_index=True,how='inner')
df_train_id['label_x']=df_train_id.loc[:,'label_x_d'].apply(lambda x:is_center(x,d_x))
df_train_id['label_y']=df_train_id.loc[:,'label_y_d'].apply(lambda y:is_center(y,d_y))
df_train_id['label']= np.array(df_train_id['label_x']) & np.array(df_train_id['label_y'])

In [58]:
#generate a feature vector for each route
def generate_dataset(data):
    df=data.drop(['trajectory_id','time_entry',
                  'time_exit','vmax','vmin','vmean','x_entry','y_entry','p_time_dur',
                  'x_exit','y_exit'],axis=1)
    df=df.drop_duplicates()
    return df

In [59]:
dframe_train_id=generate_dataset(df_train_id)
dframe_test_id=generate_dataset(df_test_id)
dframe_train_id.shape,dframe_test_id.shape

((66252, 47), (16490, 42))

In [60]:
dframe_train_id.head()

,hash,id_count,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,y_zero_prevel,start_in_center_x,start_in_center_y,is_in_center_x,is_in_center_y,label_x_d,label_y_d,label_x,label_y,label
0,0000a8602cf2def930488dee7cdad104_1,6,478.000000,6069.138638,187853.203408,5956.876032,12928.234105,16.033333,7,NaN,...,0,1,0,0,0,6116.062904,12572.734471,0,0,0
6,0000cf177130469eeac79f67b6bcf3df_9,4,31.150000,361.788345,995.188644,1813.564585,-2855.283569,29.266667,14,1.149404,...,0,0,1,0,1,1291.476173,-2969.680929,0,1,0
10,0001f97b99a80f18f62e2d44e54ef33d_3,2,177.483333,12722.704603,271805.888240,-7836.722072,107034.585072,45.266667,11,32.196255,...,1,0,0,1,0,-18784.993991,-126329.542090,1,0,0
29,0002974025ed311b78ad49904a31d4ea_25,3,66.066667,-1047.516408,-10551.981134,5178.752792,36411.900737,7.016667,13,NaN,...,1,0,0,0,0,4421.780254,86886.507634,0,0,0
42,00037fa9d812de1961f1e2603ad4b22b_25,4,152.700000,4417.157395,31119.254525,-18050.285352,-119615.536787,11.766667,12,NaN,...,0,0,0,1,0,-17511.499017,-112107.285920,1,0,0


In [61]:
df_train_id.columns

Index(['hash', 'trajectory_id', 'time_entry', 'time_exit', 'vmax', 'vmin',
       'vmean', 'x_entry', 'y_entry', 'x_exit', 'y_exit', 'p_time_dur',
       'id_count', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'vel_x', 'vel_y', 'first_dist_x', 'first_dist_y',
       'first_vx', 'first_vy', 'pre_v1_x', 'pre_v1_y', 'theta',
       'x_zero_prevel', 'y_zero_prevel', 'start_in_center_x',
       'start_in_center_y', 'is_in_center_x', 'is_in_center_y', 'label_x_d',
       'label_y_d', 'label_x', 'label_y', 'label'],
      dtype='object')

In [62]:
d_Xx=dframe_train_id[['id_count', 'feature_1', 'feature_2', 'feature_3','feature_4','feature_5','feature_6', 
        'feature_8', 'feature_9','feature_10','feature_11','feature_12',
        'feature_13', 'feature_14', 
        'feature_15', 'feature_16', 'feature_17', 
        'feature_20', 'feature_21',
        'theta','vel_x',  'pre_v1_x','vel_y','pre_v1_y',
       'x_zero_prevel','y_zero_prevel',
        'first_dist_x','first_dist_y'
        ]]
d_Xy=dframe_train_id[['id_count', 'feature_1', 'feature_2', 'feature_3','feature_4','feature_5','feature_6', 
        'feature_8', 'feature_9',
        'feature_10','feature_11','feature_12',
        'feature_13', 'feature_14', 
        'feature_15', 'feature_16', 'feature_17', 
        'feature_18', 'feature_19', 
        'feature_20', 'feature_21',
        'feature_22', 'feature_24', 
        'feature_23','feature_25',
        'theta','vel_x',  'pre_v1_x','vel_y','pre_v1_y',
       'x_zero_prevel','y_zero_prevel',
        'start_in_center_x', 'start_in_center_y',
        'first_dist_x','first_dist_y'
                     ]]
d_yx=dframe_train_id['label_x']
d_yy=dframe_train_id['label_y']
d_Xxt=dframe_test_id[['id_count', 'feature_1', 'feature_2', 'feature_3','feature_4','feature_5','feature_6', 
        'feature_8', 'feature_9','feature_10','feature_11','feature_12',
        'feature_13', 'feature_14', 
        'feature_15', 'feature_16', 'feature_17', 
        'feature_20', 'feature_21', 
        'theta','vel_x',  'pre_v1_x','vel_y','pre_v1_y',
       'x_zero_prevel','y_zero_prevel',
        'first_dist_x','first_dist_y'
                     ]]
d_Xyt=dframe_test_id[['id_count', 'feature_1', 'feature_2', 'feature_3','feature_4','feature_5','feature_6', 
        'feature_8', 'feature_9',
        'feature_10','feature_11','feature_12',
        'feature_13', 'feature_14', 
        'feature_15', 'feature_16', 'feature_17', 
        'feature_18', 'feature_19', 
        'feature_20', 'feature_21',
        'feature_22', 'feature_24', 
        'feature_23','feature_25',
        'theta','vel_x',  'pre_v1_x','vel_y','pre_v1_y',
       'x_zero_prevel','y_zero_prevel',
        'start_in_center_x', 'start_in_center_y',
        'first_dist_x','first_dist_y'
                     ]]

In [63]:
Xx_train,Xx_test,yx_train,yx_test=train_test_split(d_Xx,d_yx,test_size=0.25,stratify=d_yx,random_state=666)
Xy_train,Xy_test,yy_train,yy_test=train_test_split(d_Xy,d_yy,test_size=0.25,stratify=d_yy,random_state=666)

In [64]:
model_x=xgb.XGBClassifier(n_estimators=70,
                         max_depth=8,
                         learning_rate=0.08,
                         subsample=0.8,
                         colsample_bytree=1,
                         colsample_bylevel=1,
                         early_stopping_rounds=5)
model_x.fit(Xx_train,yx_train,eval_metric='error',eval_set=[(Xx_test,yx_test)],verbose=True)

[0]	validation_0-error:0.147558
[1]	validation_0-error:0.146652
[2]	validation_0-error:0.144962
[3]	validation_0-error:0.14309
[4]	validation_0-error:0.14303
[5]	validation_0-error:0.140675
[6]	validation_0-error:0.140977
[7]	validation_0-error:0.141218
[8]	validation_0-error:0.140977
[9]	validation_0-error:0.141218
[10]	validation_0-error:0.140977
[11]	validation_0-error:0.140735
[12]	validation_0-error:0.140916
[13]	validation_0-error:0.141037
[14]	validation_0-error:0.14146
[15]	validation_0-error:0.140735
[16]	validation_0-error:0.140494
[17]	validation_0-error:0.13983
[18]	validation_0-error:0.13983
[19]	validation_0-error:0.139588
[20]	validation_0-error:0.139166
[21]	validation_0-error:0.139286
[22]	validation_0-error:0.139286
[23]	validation_0-error:0.138924
[24]	validation_0-error:0.139649
[25]	validation_0-error:0.138562
[26]	validation_0-error:0.13832
[27]	validation_0-error:0.138381
[28]	validation_0-error:0.13832
[29]	validation_0-error:0.13826
[30]	validation_0-error:0.13

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=5, gamma=0,
       learning_rate=0.08, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=70, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [65]:
model_y=xgb.XGBClassifier(n_estimators=70,
                         max_depth=9,
                         learning_rate=0.08,
                         subsample=0.8,
                         colsample_bytree=1,
                         colsample_bylevel=1,
                         early_stopping_rounds=5)
model_y.fit(Xy_train,yy_train,eval_metric='error',eval_set=[(Xy_test,yy_test)],verbose=True)

[0]	validation_0-error:0.160237
[1]	validation_0-error:0.156735
[2]	validation_0-error:0.154259
[3]	validation_0-error:0.152569
[4]	validation_0-error:0.150878
[5]	validation_0-error:0.152146
[6]	validation_0-error:0.151603
[7]	validation_0-error:0.149973
[8]	validation_0-error:0.149369
[9]	validation_0-error:0.150939
[10]	validation_0-error:0.150275
[11]	validation_0-error:0.150516
[12]	validation_0-error:0.150577
[13]	validation_0-error:0.150395
[14]	validation_0-error:0.150878
[15]	validation_0-error:0.14949
[16]	validation_0-error:0.149309
[17]	validation_0-error:0.149309
[18]	validation_0-error:0.150154
[19]	validation_0-error:0.149852
[20]	validation_0-error:0.149611
[21]	validation_0-error:0.149731
[22]	validation_0-error:0.149309
[23]	validation_0-error:0.149248
[24]	validation_0-error:0.149007
[25]	validation_0-error:0.148765
[26]	validation_0-error:0.148463
[27]	validation_0-error:0.148946
[28]	validation_0-error:0.148946
[29]	validation_0-error:0.149128
[30]	validation_0-err

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=5, gamma=0,
       learning_rate=0.08, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=70, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [66]:
pred_x=pd.Series(model_x.predict(Xx_test))
pred_y=pd.Series(model_y.predict(Xy_test))
acc_x=metrics.f1_score(yx_test,pred_x)
acc_y=metrics.f1_score(yy_test,pred_y)
acc_x,acc_y

(0.89678589701846589, 0.75831751423718663)

In [67]:
p_label= (np.array(pred_x)) & (np.array(pred_y))
t_label= (np.array(yx_test)) & (np.array(yy_test))
acc=metrics.f1_score(t_label,p_label)
acc

0.68669260700389101

In [68]:
pred_x_t=pd.Series(model_x.predict(d_Xxt))
pred_y_t=pd.Series(model_y.predict(d_Xyt))
label_t= (np.array(pred_x_t)) & (np.array(pred_y_t))
gb=data_test_nz_id.groupby(['hash'])
last_id=gb['trajectory_id'].apply(lambda x:np.array(x)[-1])
result_m=pd.Series(label_t,index=last_id.values)
result_m.to_frame(name='city_center')
result_m

traj_00032f51796fd5437b238e3a9823d13d_31_5     0
traj_000479418b5561ab694a2870cc04fd43_25_10    0
traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4      0
traj_00063a4f6c12e1e4de7d876580620667_3_4      0
traj_000d479078af08618bddc7f09082b8c3_11_6     0
traj_000efd6f95db39d6ba35027f02b5628f_9_20     1
traj_000feace3a33be9245e2783cdff467a8_15_12    0
traj_0011b48d0cf4f962833edd701afaf419_3_7      0
traj_001484cf0f956c316070be4b4c6352ff_29_7     0
traj_001d29060cfe35996cfeb141210ecbeb_9_2      1
traj_001dff40eab7eea1c10d0fadcea0c779_9_7      0
traj_001f9345a2979c553125c65499630f25_29_3     0
traj_00265578639bd4d35a3524a1463d98bb_31_8     0
traj_00351550555ffcb00b52d21bcaabfadf_31_11    0
traj_003539963b1eebf6088d3d5302e45232_31_12    1
traj_003a9e30d316e8fc524ac36f9deac690_1_5      1
traj_003f763a38314d7b516a5561f7747198_25_2     0
traj_0042bdd6496743d4b0069beefa230dd8_5_10     0
traj_00437531cc06bdbd7c969bb3a07be14f_3_2      0
traj_0046a336a7487550daf0cb3680e15af3_11_7     0
traj_00531d7f45c3c6a

In [90]:
result=pd.concat([result_z,result_1,result_m]).to_frame(name='target')
gd=data_test.groupby(['hash'])
lid=gd['trajectory_id'].apply(lambda x:np.array(x)[-1])
res=pd.DataFrame(np.zeros(len(lid)),index=lid.values,columns=['target'])
df=pd.merge(res,result,how='inner',left_index=True,right_index=True)
df.drop(['target_x'],axis=1,inplace=True)
df.reset_index()
df.rename(columns={'index':'id','target_y':'target'})
df.to_csv('final_result.csv')
df